# Supervised Fine-Tuning with SFTTrainer

This notebook demonstrates how to fine-tune the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer` from the `trl` library. The notebook cells run and will finetune the model. You can select your difficulty by trying out different datasets.

<div style='background-color: lightblue; padding: 10px; border-radius: 5px; margin-bottom: 20px'>
    <h2 style='margin: 0;color:blue'>Exercise: Fine-Tuning SmolLM2 with SFTTrainer</h2>
    <p>Take a dataset from the Hugging Face hub and finetune a model on it. </p> 
    <p><b>Difficulty Levels</b></p>
    <p>🐢 Use the `HuggingFaceTB/smoltalk` dataset</p>
    <p>🐕 Try out the `bigcode/the-stack-smol` dataset and finetune a code generation model on a specific subset `data/python`.</p>
    <p>🦁 Select a dataset that relates to a real world use case your interested in</p>
</div>

In [1]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path=model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)


## Dataset Preparation

We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

In [2]:
# Load a sample dataset
from datasets import load_dataset

# TODO: define your dataset and config using the path and name parameters
ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

In [3]:
# TODO: 🦁 If your dataset is not in a 'chatml' format, you will need to process it. Refer to the [module](../chat_templates.md)

## Configuring the SFTTrainer

The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [4]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=1000,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=ds["train"],
    tokenizer=tokenizer,
    eval_dataset=ds["test"],
)

# TODO: 🦁 🐕 align the SFTTrainer params with your chosen dataset. For example, if you are using the `bigcode/the-stack-smol` dataset, you will need to choose the `content` column`

/home/duydl/Miniconda3/envs/py310_kotaemon/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/duydl/Miniconda3/envs/py310_kotaemon/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


## Training the Model

With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [5]:
# Train the model
trainer.train()

# Save the model
trainer.save_model("./sft_output")

  0%|          | 0/1000 [00:00<?, ?it/s]

{'loss': 1.6774, 'grad_norm': 2.7205302715301514, 'learning_rate': 4.9500000000000004e-05, 'epoch': 0.02}
{'loss': 1.2563, 'grad_norm': 2.644501209259033, 'learning_rate': 4.9e-05, 'epoch': 0.04}
{'loss': 1.234, 'grad_norm': 2.394712448120117, 'learning_rate': 4.85e-05, 'epoch': 0.05}
{'loss': 1.161, 'grad_norm': 2.5028374195098877, 'learning_rate': 4.8e-05, 'epoch': 0.07}
{'loss': 1.0657, 'grad_norm': 2.2998411655426025, 'learning_rate': 4.75e-05, 'epoch': 0.09}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.1589809656143188, 'eval_runtime': 2.0524, 'eval_samples_per_second': 57.982, 'eval_steps_per_second': 7.309, 'epoch': 0.09}
{'loss': 1.1566, 'grad_norm': 2.2886111736297607, 'learning_rate': 4.7e-05, 'epoch': 0.11}
{'loss': 1.1648, 'grad_norm': 2.012437582015991, 'learning_rate': 4.6500000000000005e-05, 'epoch': 0.12}
{'loss': 1.1244, 'grad_norm': 2.1284937858581543, 'learning_rate': 4.600000000000001e-05, 'epoch': 0.14}
{'loss': 1.086, 'grad_norm': 2.2243947982788086, 'learning_rate': 4.55e-05, 'epoch': 0.16}
{'loss': 1.1116, 'grad_norm': 2.0808310508728027, 'learning_rate': 4.5e-05, 'epoch': 0.18}


  0%|          | 0/15 [00:00<?, ?it/s]

...

{'eval_loss': 1.022836446762085, 'eval_runtime': 1.961, 'eval_samples_per_second': 60.684, 'eval_steps_per_second': 7.649, 'epoch': 1.59}
{'loss': 0.8331, 'grad_norm': 1.7076342105865479, 'learning_rate': 4.5e-06, 'epoch': 1.61}
{'loss': 0.8059, 'grad_norm': 1.8049014806747437, 'learning_rate': 4.000000000000001e-06, 'epoch': 1.63}
{'loss': 0.8107, 'grad_norm': 2.0719687938690186, 'learning_rate': 3.5000000000000004e-06, 'epoch': 1.65}
{'loss': 0.7932, 'grad_norm': 1.7175118923187256, 'learning_rate': 3e-06, 'epoch': 1.66}
{'loss': 0.8623, 'grad_norm': 1.9925297498703003, 'learning_rate': 2.5e-06, 'epoch': 1.68}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0216712951660156, 'eval_runtime': 1.9589, 'eval_samples_per_second': 60.749, 'eval_steps_per_second': 7.657, 'epoch': 1.68}
{'loss': 0.8609, 'grad_norm': 1.8237707614898682, 'learning_rate': 2.0000000000000003e-06, 'epoch': 1.7}
{'loss': 0.7802, 'grad_norm': 1.7281429767608643, 'learning_rate': 1.5e-06, 'epoch': 1.72}
{'loss': 0.7957, 'grad_norm': 1.7798877954483032, 'learning_rate': 1.0000000000000002e-06, 'epoch': 1.73}
{'loss': 0.8345, 'grad_norm': 1.7968848943710327, 'learning_rate': 5.000000000000001e-07, 'epoch': 1.75}
{'loss': 0.7915, 'grad_norm': 1.764425277709961, 'learning_rate': 0.0, 'epoch': 1.77}


  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 1.0214853286743164, 'eval_runtime': 1.9642, 'eval_samples_per_second': 60.584, 'eval_steps_per_second': 7.637, 'epoch': 1.77}
{'train_runtime': 314.6141, 'train_samples_per_second': 12.714, 'train_steps_per_second': 3.178, 'train_loss': 0.9605711579322815, 'epoch': 1.77}


## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `SFTTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.